In [1]:
from pathlib import Path

from dataset_loading import load_glue_sentence_classification

In [2]:
datasets = load_glue_sentence_classification(Path("../GLUE-baselines/glue_data/SST-2/"))

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 
def tokenize_function(sample):
  return tokenizer(sample["sentence"], max_length=200, padding="max_length", truncation=True)


In [4]:
tokenized_datasets = datasets.map(tokenize_function,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence"])
tokenized_datasets["train"] = tokenized_datasets["train"].rename_column("label","labels")
tokenized_datasets["dev"] = tokenized_datasets["dev"].rename_column("label","labels")
tokenized_datasets.set_format("torch")

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [5]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

c:\Python310\lib\site-packages\torch\serialization.py:871: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  obj = cast(Storage, torch._UntypedStorage(nbytes))


AssertionError: 

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets["dev"], shuffle=True, batch_size=8)
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
   for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        print(batch)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/25257 [00:00<?, ?it/s]

{'labels': tensor([1, 1, 0, 1, 0, 1, 1, 1]), 'input_ids': tensor([[  101,  8052,  1999,  ...,     0,     0,     0],
        [  101,  1037,  4072,  ...,     0,     0,     0],
        [  101, 21741,  2015,  ...,     0,     0,     0],
        ...,
        [  101,  1996, 12127,  ...,     0,     0,     0],
        [  101, 18276,  2003,  ...,     0,     0,     0],
        [  101,  4654, 21436,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'labels': tensor([1, 0, 1, 1, 1, 0, 0, 0]), 'input_ids': tensor([[  101,  6118,   102,  ...,     0,     

KeyboardInterrupt: 

In [ ]:
model.eval()

num = 0
correct = 0
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    #print(logits)
    #print(predictions)
    #print(batch["labels"])
    correct+=torch.sum(predictions==batch["labels"])
    num+=len(predictions)
    print(correct/num)


tensor(0.6250)
tensor(0.5000)
tensor(0.4583)
tensor(0.4062)
tensor(0.3500)
tensor(0.4167)
tensor(0.4286)
tensor(0.4531)
tensor(0.4583)
tensor(0.4750)
tensor(0.4773)
tensor(0.4688)
tensor(0.4808)
tensor(0.4821)
tensor(0.4667)
tensor(0.4688)
tensor(0.4853)
tensor(0.4931)
tensor(0.5066)
tensor(0.4938)
tensor(0.4881)
tensor(0.4830)
tensor(0.4728)
tensor(0.4635)
tensor(0.4750)
tensor(0.4760)
tensor(0.4676)
tensor(0.4777)
tensor(0.4741)
tensor(0.4750)
tensor(0.4839)
tensor(0.4883)
tensor(0.4811)
tensor(0.4779)
tensor(0.4821)
tensor(0.4826)
tensor(0.4797)
tensor(0.4770)
tensor(0.4744)
tensor(0.4781)
tensor(0.4848)
tensor(0.4851)
tensor(0.4942)
tensor(0.4943)
tensor(0.4917)
tensor(0.4918)
tensor(0.4947)
tensor(0.4896)
tensor(0.4923)
tensor(0.4950)
tensor(0.4902)
tensor(0.4928)
tensor(0.4882)
tensor(0.4884)
tensor(0.4864)
tensor(0.4911)
tensor(0.4912)


KeyboardInterrupt: 